In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [3]:
train=pd.read_csv('../Input/train.csv')
test=pd.read_csv('../Input/test.csv')

In [8]:
new_condition_non_zero_odometer=train[(train['condition']=="new") & (train['odometer']>0)]

new_condition_non_zero_odometer_count = len(new_condition_non_zero_odometer)
new_condition_non_zero_odometer_count

54

In [ ]:
train['price'] = np.log(train['price'])

In [ ]:
def preprocess(df):
    # 2999年以上のものは-1000するように
    df.loc[df['year'] >= 2999, 'year'] = df.loc[df['year'] >= 2999, 'year'] - 1000
    #cylinders列を数値型に
    df['cylinders'] = df['cylinders'].str.extract('(\d+)').astype(float)
    df['cylinders'] = df['cylinders'].replace('nan', 2.0)
    df['cylinders']=df['cylinders'].astype(float)
    
    #年間走行距離
    df['odometer_per_year'] = df['odometer'] / ((df['year'].max() + 1) - df['year'])
    #クレンジング
    df['manufacturer_cleaned'] = df['manufacturer'].apply(clean_manufacturer)
    #車の経過年数
    df['car_age'] = (df['year'].max() + 1) - df['year']
    #paint_colorをワンホットエンコーディング
    df = pd.get_dummies(df, columns=['paint_color'])
    #fuel列の欠損値をgasで補完
    df['fuel'] = df['fuel'].fillna('gas')
    #fuel列をラベルエンコーディング
    df['fuel'] = label_encoder.fit_transform(df['fuel'])
    df.fillna('nan', inplace=True)
    
    for c in ['region','manufacturer_cleaned','condition','fuel','title_status','transmission', 'drive', 'size', 'type', 'state']:
        df[c] = label_encoder.fit_transform(df[c].astype(str))
    #不要な列を削除
    del df['manufacturer']
    del df['type']
    return df